In [1]:
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random


SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

torch.backends.cudnn.benchmark = True


Device: cuda


In [2]:

import os

BASE_DIR = os.getcwd()   

csv_path = os.path.join(BASE_DIR, "pems_bay_final_with_extra_features.csv")
adj_path = os.path.join(BASE_DIR, "adj_mx_PEMS-BAY.pkl")


if not os.path.exists(csv_path):
    raise FileNotFoundError(f"CSV not found: {csv_path}")

if not os.path.exists(adj_path):
    raise FileNotFoundError(f"Adjacency file not found: {adj_path}")


print("CSV Path:", csv_path)
print("Adj Path:", adj_path)


CSV Path: C:\Users\akanksh_02\Downloads\trf\pems_bay_final_with_extra_features.csv
Adj Path: C:\Users\akanksh_02\Downloads\trf\adj_mx_PEMS-BAY.pkl


In [3]:

df = pd.read_csv(
    csv_path,
    index_col="timestamp",
    parse_dates=True,
    low_memory=False
)

print("Dataset shape:", df.shape)
print("Columns:", len(df.columns))



for col in df.columns:
    if df[col].dtype == "float64":
        df[col] = df[col].astype("float32")

print("Memory optimized ✓")



📂 Loading CSV...
Dataset shape: (52116, 338)
Columns: 338
Memory optimized ✓


In [4]:

print("\n Selecting sensor + time features...")


sensor_cols = [c for c in df.columns if c.isdigit()]

if len(sensor_cols) == 0:
    raise ValueError("No sensor columns detected!")

print("Number of sensors (nodes):", len(sensor_cols))



time_cols = [
    "hour_sin", "hour_cos",
    "dow_sin", "dow_cos",
    "weekend", "holiday"
]

for c in time_cols:
    if c not in df.columns:
        raise ValueError(f"Missing time feature column: {c}")



traffic = df[sensor_cols].to_numpy(dtype=np.float32)
time_feat = df[time_cols].to_numpy(dtype=np.float32)


print("Traffic shape     :", traffic.shape)   # (T, N)
print("Time feat shape   :", time_feat.shape) # (T, F_time)



🧩 Selecting sensor + time features...
Number of sensors (nodes): 325
Traffic shape     : (52116, 325)
Time feat shape   : (52116, 6)


In [5]:


print("\n📊 Normalizing traffic per sensor...")

# compute mean/std for EACH sensor (column-wise)
mean = traffic.mean(axis=0, keepdims=True)
std  = traffic.std(axis=0, keepdims=True)

# avoid divide by zero
std[std == 0] = 1.0

traffic = (traffic - mean) / std

traffic = traffic.astype(np.float32)

print("Normalized ✓")
print("Mean shape:", mean.shape)  # (1, N)
print("Std shape :", std.shape)



📊 Normalizing traffic per sensor...
Normalized ✓
Mean shape: (1, 325)
Std shape : (1, 325)


In [6]:

print("\n Loading adjacency...")

with open(adj_path, "rb") as f:
    adj_data = pickle.load(f, encoding="latin1")

A = adj_data[2].astype(np.float32)

print("Raw adjacency:", A.shape)



A = A + np.eye(A.shape[0], dtype=np.float32)



D = np.sum(A, axis=1)
D_inv_sqrt = np.diag(1.0 / np.sqrt(D + 1e-8))

A_norm = D_inv_sqrt @ A @ D_inv_sqrt



adj_mx = torch.tensor(A_norm, dtype=torch.float32).to(device)

print("Normalized adjacency shape:", adj_mx.shape)



🕸 Loading adjacency...
Raw adjacency: (325, 325)
Normalized adjacency shape: torch.Size([325, 325])


In [7]:

print("\n🔗 Combining traffic + time features...")

T, N = traffic.shape
F_time = time_feat.shape[1]


traffic = traffic[..., None]          # (T, N, 1)

time_feat_expanded = np.broadcast_to(
    time_feat[:, None, :],            # (T,1,F_time)
    (T, N, F_time)                   # (T,N,F_time)
)


data = np.concatenate(
    [traffic, time_feat_expanded],
    axis=2
).astype(np.float32)


print("Time steps (T):", T)
print("Nodes (N):", N)
print("Features per node:", data.shape[2])
print("Final data shape:", data.shape)



🔗 Combining traffic + time features...
Time steps (T): 52116
Nodes (N): 325
Features per node: 7
Final data shape: (52116, 325, 7)


In [8]:

SEQ_LEN = 24
PRED_LEN = 3


class TrafficDataset(Dataset):

    def __init__(self, data):
        self.data = data.astype(np.float32)

    def __len__(self):
        return len(self.data) - SEQ_LEN - PRED_LEN

    def __getitem__(self, idx):

        x = self.data[idx : idx+SEQ_LEN]                 # (T,N,F)
        y = self.data[idx+SEQ_LEN : idx+SEQ_LEN+PRED_LEN, :, 0]  # (P,N)

       
        x = torch.from_numpy(x).permute(2,1,0)  # (F,N,T)
        y = torch.from_numpy(y)

        return x, y


In [9]:


print("\n📦 Creating train/test split...")

split = int(len(data) * 0.8)

train_data = data[:split]
test_data  = data[split:]

print("Train samples:", len(train_data))
print("Test samples :", len(test_data))


BATCH_SIZE = 64



train_loader = DataLoader(
    TrafficDataset(train_data),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,      
    pin_memory=True,    
    drop_last=True      
)

test_loader = DataLoader(
    TrafficDataset(test_data),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

print("Batches per epoch:", len(train_loader))



📦 Creating train/test split...
Train samples: 41692
Test samples : 10424
Batches per epoch: 651


In [10]:


class GraphWaveNetDeep(nn.Module):
    

    def __init__(self, num_nodes, in_dim, out_dim, adj):
        super().__init__()

        self.adj = adj

        channels = 64
        num_blocks = 6   # deep network

     
        self.input_proj = nn.Conv2d(in_dim, channels, kernel_size=(1,1))


        self.temporal_layers = nn.ModuleList()
        self.graph_layers = nn.ModuleList()

        for _ in range(num_blocks):

            # SAME padding (no shrink)
            self.temporal_layers.append(
                nn.Conv2d(
                    channels,
                    channels,
                    kernel_size=(1,3),
                    padding=(0,1)
                )
            )

            self.graph_layers.append(
                nn.Linear(num_nodes, num_nodes, bias=False)
            )


        self.output_proj = nn.Conv2d(channels, out_dim, kernel_size=(1,1))


  
    def graph_conv(self, x):
        return torch.einsum("bfnt,nm->bfmt", x, self.adj)


   
    def forward(self, x):

        x = self.input_proj(x)

        for tconv, _ in zip(self.temporal_layers, self.graph_layers):

            residual = x

            # temporal learning
            x = F.relu(tconv(x))

            # spatial learning
            x = self.graph_conv(x)

            # residual
            x = x + residual


        x = self.output_proj(x)

        # average over time dimension
        return x.mean(dim=-1)


In [11]:


print("\n🧠 Initializing Deep GraphWaveNet model...")

in_dim = data.shape[2]   # features per node
num_nodes = N
out_dim = PRED_LEN

print("Input features:", in_dim)
print("Nodes:", num_nodes)
print("Prediction horizon:", out_dim)


model = GraphWaveNetDeep(
    num_nodes=num_nodes,
    in_dim=in_dim,
    out_dim=out_dim,
    adj=adj_mx
).to(device)



total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model parameters: {total_params/1e6:.2f}M")
print("Model device:", next(model.parameters()).device)
print("Model ready ✓\n")



🧠 Initializing Deep GraphWaveNet model...
Input features: 7
Nodes: 325
Prediction horizon: 3
Model parameters: 0.71M
Model device: cuda:0
Model ready ✓



In [12]:


from tqdm import tqdm   # progress bar

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

EPOCHS = 25



train_loader = DataLoader(
    TrafficDataset(train_data),
    batch_size=64,
    shuffle=True,
    num_workers=0,      # ⭐ IMPORTANT for Windows
    pin_memory=True,
    drop_last=True
)


print("\n🚀 Training Deep GraphWaveNet...\n")


for epoch in range(EPOCHS):

    model.train()
    epoch_loss = 0

    # tqdm progress bar
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=True)

    for x, y in loop:

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        pred = model(x)
        loss = criterion(pred, y)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # ⭐ show live loss
        loop.set_postfix(loss=loss.item())


    print(f"✅ Epoch {epoch+1}/{EPOCHS} Loss: {epoch_loss/len(train_loader):.4f}\n")



🚀 Training Deep GraphWaveNet...



Epoch 1/25: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 651/651 [03:31<00:00,  3.08it/s, loss=0.262]


✅ Epoch 1/25 Loss: 0.3130



Epoch 2/25: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 651/651 [03:39<00:00,  2.97it/s, loss=0.236]


✅ Epoch 2/25 Loss: 0.2421



...

✅ Epoch 23/25 Loss: 0.1587



Epoch 24/25: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 651/651 [03:17<00:00,  3.29it/s, loss=0.178]


✅ Epoch 24/25 Loss: 0.1582



Epoch 25/25: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 651/651 [03:10<00:00,  3.41it/s, loss=0.155]

✅ Epoch 25/25 Loss: 0.1583



In [27]:

print("\n📊 Evaluating on test set...\n")

model.eval()

mae_sum = 0
mse_sum = 0
count = 0

with torch.no_grad():

    for x, y in test_loader:

        x = x.to(device)
        y = y.to(device)

        pred = model(x)

        mae_sum += torch.abs(pred - y).sum().item()
        mse_sum += ((pred - y) ** 2).sum().item()
        count += y.numel()


mae_norm = mae_sum / count
rmse_norm = (mse_sum / count) ** 0.5

real_mae = mae_norm * std.mean()
real_rmse = rmse_norm * std.mean()


print("===================================")
print("Normalized MAE :", round(mae_norm, 4))
print("Normalized RMSE:", round(rmse_norm, 4))
print("-----------------------------------")
print("Real MAE :", round(real_mae, 3))
print("Real RMSE:", round(real_rmse, 3))
print("===================================")



📊 Evaluating on test set...

Normalized MAE : 0.1618
Normalized RMSE: 0.3295
-----------------------------------
Real MAE : 1.385
Real RMSE: 2.819


In [29]:
torch.save(model.state_dict(), "deep_graphwavenet.pth")
print("Model saved ✓")


Model saved ✓


In [37]:
from sklearn.metrics import r2_score

preds = []
trues = []

with torch.no_grad():
    for x,y in test_loader:
        x,y = x.to(device), y.to(device)
        pred = model(x)

        preds.append(pred.cpu().numpy().ravel())
        trues.append(y.cpu().numpy().ravel())

preds = np.concatenate(preds)
trues = np.concatenate(trues)

print("R² score:", round(r2_score(trues, preds), 4))


R² score: 0.8932
